In [1]:
#librerias del proyecto
from sqlalchemy import create_engine
import pyodbc
import numpy as np
import re 
import pandas as pd 
import datetime
from datetime import date

In [2]:
# conecion a base de datos
try:
    cnxn = pyodbc.connect('DSN=CARGA_BI;UID=xxxx;PWD=*****')
    cursor = cnxn.cursor()
    hora = datetime.datetime.now ()
    with open('Operacion_vt11.txt', 'a') as fobj:
        fobj.write(str(hora)+'conexion exitosa \n')
except BaseException as error:
    hora = datetime.datetime.now ()
    with open('Operacion_vt11.txt', 'a') as fobj:
        fobj.write(str(hora)+'conexion BBDD error: '+ str(error)+' \n' )     


In [3]:
vt11=pd.read_sql(""" select OPE_INDICADORES_PLANTA.transporte transporte,
          OPE_INDICADORES_PLANTA.[Creado por] Creado_por,
          OPE_INDICADORES_PLANTA.[Placa T] Placa_T,
          OPE_INDICADORES_PLANTA.Piloto Piloto,
          OPE_INDICADORES_PLANTA.Licencia Licencia,
          OPE_INDICADORES_PLANTA.inPlanCarg inPlanCarg,
          OPE_INDICADORES_PLANTA.horaplreg horaplreg,
          OPE_INDICADORES_PLANTA.HoraAcRg HoraAcRg,
          OPE_INDICADORES_PLANTA.HrPrIC HrPrIC,
          OPE_INDICADORES_PLANTA.HrAiCg HrAiCg,
          OPE_INDICADORES_PLANTA.Cump Cump0,
          OPE_INDICADORES_PLANTA.HrPrFiCga HrPrFiCga,
          OPE_INDICADORES_PLANTA.HrAFCg HrAFCg,
          OPE_INDICADORES_PLANTA.Cum Cum,
          OPE_INDICADORES_PLANTA.HoraPlDS HoraPlDS,
          OPE_INDICADORES_PLANTA.HrADpEXP HrADpEXP,
          OPE_INDICADORES_PLANTA.OTIF OTIF,
          OPE_INDICADORES_PLANTA.[Plan T Car] Plan_T_Car,
          OPE_INDICADORES_PLANTA.[Real T Car] Real_T_Car,
          OPE_INDICADORES_PLANTA.[Peso Teori] Peso_Teori,
          OPE_INDICADORES_PLANTA.[pESO rEAL] pESO_rEAL,
          OPE_INDICADORES_PLANTA.eNTREGAS eNTREGAS,
          OPE_INDICADORES_PLANTA.PTRP PTRP,
          OPE_INDICADORES_PLANTA.PLAn_TIEMPO_CARGA PLAn_TIEMPO_CARGA,
          OPE_INDICADORES_PLANTA.REAL_TIEMPO_CARGA REAL_TIEMPO_CARGA,
          OPE_INDICADORES_PLANTA.PLANTA PLANTA,
          OPE_INDICADORES_PLANTA.PAIS PAIS,
          OPE_INDICADORES_PLANTA.Cump Cump,
          OPE_INDICADORES_PLANTA.Cumple_tiempo_carga Cumple_tiempo_carga,
          OPE_INDICADORES_PLANTA.Cumple_inicio_carga Cumple_inicio_carga,
           OPE_INDICADORES_PLANTA.cumple_cliente cumple_cliente, 
    OPE_INDICADORES_PLANTA.Demora_tiempo_carga_R_C,DIF_PROM_INI_CARGA 
 ,  B.OTIF OBJETIVO_OTIF,
          B.CUMP_H_INI_CARGA  OBJETIVO_CUMP_H_INI_CARGA ,
          B.DEMORA_PROME_H_INI_CARGA OBJETIVO_DEMORA_PROME_H_INI_CARGA,
          B.CUMP_T_CARAGA OBJETIVO_CUMP_T_CARAGA,
          B.DEMORA_PROME_T_CARGA OBJETIVO_DEMORA_PROME_T_CARGA,
          B.CUMPLE_CLIENTE OBJETIVO_CUMPLE_CLIENTE
from   dbo.OPE_INDICADORES_PLANTA OPE_INDICADORES_PLANTA   LEFT JOIN    dbo.OPE_PLANTA_PLAN B 
ON (  OPE_INDICADORES_PLANTA.PTRP=B.PTRP)""", con=cnxn, parse_dates=True )

In [4]:
# limpieza de nulos
vt11['Creado_por'].fillna('', inplace=True)
vt11['Placa_T'].fillna('', inplace=True)
vt11['Piloto'].fillna('', inplace=True)
vt11['Licencia'].fillna('', inplace=True)
vt11['Creado_por']=vt11['Creado_por'].astype('str')
vt11['Placa_T']=vt11['Placa_T'].astype('str')
vt11['Piloto']=vt11['Piloto'].astype('str')
vt11['Licencia']=vt11['Licencia'].astype('str')

In [5]:
# Convert account_opened to datetime
vt11['inPlanCarg'] = pd.to_datetime(vt11['inPlanCarg'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 

In [6]:
#OTIF
VT11_OTIF=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['OTIF','OBJETIVO_OTIF']].mean()
VT11_OTIF=VT11_OTIF.reset_index()
VT11_OTIF['INDICADOR']='OTIF'
VT11_OTIF['UNIDAD']='%'
VT11_OTIF['OTIF']=VT11_OTIF['OTIF']*100
VT11_OTIF['OBJETIVO_OTIF']=VT11_OTIF['OBJETIVO_OTIF']*100
VT11_OTIF['STATUS']=VT11_OTIF['OTIF']/VT11_OTIF['OBJETIVO_OTIF']*100

In [7]:
#OTIF
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_OTIF_semana=VT11_OTIF
VT11_OTIF_semana['DIA_SEMANA']=VT11_OTIF_semana.inPlanCarg.dt.dayofweek
VT11_OTIF_semana['DIA_MES']=VT11_OTIF_semana.inPlanCarg.dt.day
VT11_OTIF_semana['ANIO']=VT11_OTIF_semana.inPlanCarg.dt.year
VT11_OTIF_semana['DIA_ANIO']=VT11_OTIF_semana.inPlanCarg.dt.dayofyear
VT11_OTIF_semana['NO_SEMANA']=VT11_OTIF_semana.inPlanCarg.dt.weekofyear
VT11_OTIF_semana['MES']=VT11_OTIF_semana.inPlanCarg.dt.month

In [8]:
#OTIF#semana
for  row_index,row  in VT11_OTIF_semana.iterrows():
    OTIF_SEMANAL=0
    v_planta=VT11_OTIF_semana['PLANTA']== VT11_OTIF_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_OTIF_semana['PAIS']==VT11_OTIF_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_OTIF_semana['NO_SEMANA']==VT11_OTIF_semana.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_OTIF_semana['DIA_SEMANA']<=VT11_OTIF_semana.loc[row_index ,'DIA_SEMANA' ]
    OTIF_SEMANAL=VT11_OTIF_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['OTIF'].mean()
    VT11_OTIF_semana.loc[row_index, 'OTIF_SEMANAL']=OTIF_SEMANAL

In [9]:
#OTIF#Mes
for  row_index,row  in VT11_OTIF_semana.iterrows():
    OTIF_MENSUAL=0
    v_planta=VT11_OTIF_semana['PLANTA']== VT11_OTIF_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_OTIF_semana['PAIS']==VT11_OTIF_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_OTIF_semana['MES']==VT11_OTIF_semana.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_OTIF_semana['DIA_MES']<=VT11_OTIF_semana.loc[row_index ,'DIA_MES' ]
    OTIF_MENSUAL=VT11_OTIF_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['OTIF'].mean()
    VT11_OTIF_semana.loc[row_index, 'OTIF_MENSUL']=OTIF_MENSUAL

In [10]:
#OTIF#anio
for  row_index,row  in VT11_OTIF_semana.iterrows():
    OTIF_ANIO=0
    v_planta=VT11_OTIF_semana['PLANTA']== VT11_OTIF_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_OTIF_semana['PAIS']==VT11_OTIF_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_OTIF_semana['ANIO']==VT11_OTIF_semana.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_OTIF_semana['DIA_ANIO']<=VT11_OTIF_semana.loc[row_index ,'DIA_ANIO' ]
    OTIF_ANIO=VT11_OTIF_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['OTIF'].mean()
    VT11_OTIF_semana.loc[row_index, 'OTIF_ANIO']=OTIF_ANIO

In [11]:
VT11_OTIF_semana['STATUS_SEMANAL']=VT11_OTIF['OTIF_SEMANAL']/VT11_OTIF['OBJETIVO_OTIF']*100
VT11_OTIF_semana['STATUS_MENSUAL']=VT11_OTIF['OTIF_MENSUL']/VT11_OTIF['OBJETIVO_OTIF']*100
VT11_OTIF_semana['STATUS_ANUAL']=VT11_OTIF['OTIF_ANIO']/VT11_OTIF['OBJETIVO_OTIF']*100
VT11_OTIF_semana.fillna(0, inplace=True)    

In [12]:
# Convert account_opened to datetime
vt11['HrAiCg'] = pd.to_datetime(vt11['HrAiCg'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 
# Convert account_opened to datetime
vt11['HrPrFiCga'] = pd.to_datetime(vt11['HrPrFiCga'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 
vt11['Cump0']=vt11['Cump0'].astype('int')
vt11[['Cump0', 'HrAiCg','HrPrFiCga']]
vt11_DIFERENCIA_CUMPLE_CLIENTE=vt11['HrPrFiCga'] -  vt11['HrAiCg'] 
vt11["DIFERENCIA_CUMPLE_CLIENTE"]=abs(vt11_DIFERENCIA_CUMPLE_CLIENTE.dt.total_seconds())
for x,y in enumerate(vt11['DIFERENCIA_CUMPLE_CLIENTE']):
    if vt11.iloc[x,10]==1: 
        vt11.iloc[x,39]=np.nan

In [13]:
#Demora en el inicio de carga --OBJETIVO_DEMORA_PROME_H_INI_CARGA
VT11_DEMORA_INICIO_CLIENTE=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['DIFERENCIA_CUMPLE_CLIENTE','OBJETIVO_DEMORA_PROME_H_INI_CARGA']].mean()
VT11_DEMORA_INICIO_CLIENTE=VT11_DEMORA_INICIO_CLIENTE.reset_index()
VT11_DEMORA_INICIO_CLIENTE['INDICADOR']='Demora en el inicio de carga'
VT11_DEMORA_INICIO_CLIENTE['UNIDAD']='Minutos'
VT11_DEMORA_INICIO_CLIENTE['DIFERENCIA_CUMPLE_CLIENTE']=VT11_DEMORA_INICIO_CLIENTE['DIFERENCIA_CUMPLE_CLIENTE']/60

In [14]:
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_DEMORA_INICIO_CLIENTE_semana=VT11_DEMORA_INICIO_CLIENTE
VT11_DEMORA_INICIO_CLIENTE_semana['DIA_SEMANA']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.dayofweek
VT11_DEMORA_INICIO_CLIENTE_semana['DIA_MES']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.day
VT11_DEMORA_INICIO_CLIENTE_semana['ANIO']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.year
VT11_DEMORA_INICIO_CLIENTE_semana['DIA_ANIO']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.dayofyear
VT11_DEMORA_INICIO_CLIENTE_semana['NO_SEMANA']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.weekofyear
VT11_DEMORA_INICIO_CLIENTE_semana['MES']=VT11_DEMORA_INICIO_CLIENTE_semana.inPlanCarg.dt.month

In [15]:
for  row_index,row  in VT11_DEMORA_INICIO_CLIENTE_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_DEMORA_INICIO_CLIENTE_semana['PLANTA']== VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_DEMORA_INICIO_CLIENTE_semana['PAIS']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_DEMORA_INICIO_CLIENTE_semana['NO_SEMANA']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_DEMORA_INICIO_CLIENTE_semana['DIA_SEMANA']<=VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_DEMORA_INICIO_CLIENTE_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['DIFERENCIA_CUMPLE_CLIENTE'].mean()
    VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index, 'DIFERENCIA_CUMPLE_CLIENTE_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

In [16]:
for  row_index,row  in VT11_DEMORA_INICIO_CLIENTE_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_DEMORA_INICIO_CLIENTE_semana['PLANTA']== VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_DEMORA_INICIO_CLIENTE_semana['PAIS']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_DEMORA_INICIO_CLIENTE_semana['MES']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_DEMORA_INICIO_CLIENTE_semana['DIA_MES']<=VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_DEMORA_INICIO_CLIENTE_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['DIFERENCIA_CUMPLE_CLIENTE'].mean()
    VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index, 'DIFERENCIA_CUMPLE_CLIENTE_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

In [17]:
for  row_index,row  in VT11_DEMORA_INICIO_CLIENTE_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_DEMORA_INICIO_CLIENTE_semana['PLANTA']== VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_DEMORA_INICIO_CLIENTE_semana['PAIS']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_DEMORA_INICIO_CLIENTE_semana['ANIO']==VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_DEMORA_INICIO_CLIENTE_semana['DIA_ANIO']<=VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_DEMORA_INICIO_CLIENTE_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['DIFERENCIA_CUMPLE_CLIENTE'].mean()
    VT11_DEMORA_INICIO_CLIENTE_semana.loc[row_index, 'DIFERENCIA_CUMPLE_CLIENTE_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    

In [18]:
 #Cumple_inicio_carga
VT11_CUMPLE_INI_CARGA=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['Cumple_inicio_carga','OBJETIVO_CUMP_H_INI_CARGA']].mean()
VT11_CUMPLE_INI_CARGA=VT11_CUMPLE_INI_CARGA.reset_index()
VT11_CUMPLE_INI_CARGA['INDICADOR']='Cumplimiento Inicio de Carga'
VT11_CUMPLE_INI_CARGA['UNIDAD']='%'
VT11_CUMPLE_INI_CARGA['Cumple_inicio_carga']=VT11_CUMPLE_INI_CARGA['Cumple_inicio_carga']*100

In [19]:
 #Cumple_inicio_carga
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_CUMPLE_INI_CARGA_semana=VT11_CUMPLE_INI_CARGA
VT11_CUMPLE_INI_CARGA_semana['DIA_SEMANA']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.dayofweek
VT11_CUMPLE_INI_CARGA_semana['DIA_MES']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.day
VT11_CUMPLE_INI_CARGA_semana['ANIO']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.year
VT11_CUMPLE_INI_CARGA_semana['DIA_ANIO']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.dayofyear
VT11_CUMPLE_INI_CARGA_semana['NO_SEMANA']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.weekofyear
VT11_CUMPLE_INI_CARGA_semana['MES']=VT11_CUMPLE_INI_CARGA_semana.inPlanCarg.dt.month

In [20]:
 #Cumple_inicio_carga
for  row_index,row  in VT11_CUMPLE_INI_CARGA_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_CUMPLE_INI_CARGA_semana['PLANTA']== VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_CUMPLE_INI_CARGA_semana['PAIS']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_CUMPLE_INI_CARGA_semana['NO_SEMANA']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_CUMPLE_INI_CARGA_semana['DIA_SEMANA']<=VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_CUMPLE_INI_CARGA_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_inicio_carga'].mean()
    VT11_CUMPLE_INI_CARGA_semana.loc[row_index, 'Cumplimiento Inicio de Carga_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

In [21]:
 #Cumple_inicio_carga
for  row_index,row  in VT11_CUMPLE_INI_CARGA_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_CUMPLE_INI_CARGA_semana['PLANTA']== VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_CUMPLE_INI_CARGA_semana['PAIS']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_CUMPLE_INI_CARGA_semana['MES']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_CUMPLE_INI_CARGA_semana['DIA_MES']<=VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_CUMPLE_INI_CARGA_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_inicio_carga'].mean()
    VT11_CUMPLE_INI_CARGA_semana.loc[row_index, 'Cumplimiento Inicio de Carga_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

In [22]:
 #Cumple_inicio_carga
for  row_index,row  in VT11_CUMPLE_INI_CARGA_semana.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_CUMPLE_INI_CARGA_semana['PLANTA']== VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PLANTA' ]
    v_pais=VT11_CUMPLE_INI_CARGA_semana['PAIS']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_CUMPLE_INI_CARGA_semana['ANIO']==VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_CUMPLE_INI_CARGA_semana['DIA_ANIO']<=VT11_CUMPLE_INI_CARGA_semana.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_CUMPLE_INI_CARGA_semana[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_inicio_carga'].mean()
    VT11_CUMPLE_INI_CARGA_semana.loc[row_index, 'Cumplimiento Inicio de Carga_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    

In [23]:
 #Cumple_inicio_carga
VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']=VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana['STATUS']=VT11_CUMPLE_INI_CARGA_semana['Cumple_inicio_carga']/VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana['STATUS_SEMANAL']=VT11_CUMPLE_INI_CARGA_semana['Cumplimiento Inicio de Carga_SEMANAL']/VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana['STATUS_MENSUAL']=VT11_CUMPLE_INI_CARGA_semana['Cumplimiento Inicio de Carga_MENSUAL']/VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana['STATUS_ANUAL']=VT11_CUMPLE_INI_CARGA_semana['Cumplimiento Inicio de Carga_ANUAL']/VT11_CUMPLE_INI_CARGA_semana['OBJETIVO_CUMP_H_INI_CARGA']*100
VT11_CUMPLE_INI_CARGA_semana.fillna(0, inplace=True)    

In [24]:
#cumple_cliente
VT11_CUMPLE_CLIENTE=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['cumple_cliente','OBJETIVO_CUMPLE_CLIENTE']].mean()
VT11_CUMPLE_CLIENTE=VT11_CUMPLE_CLIENTE.reset_index()
VT11_CUMPLE_CLIENTE['INDICADOR']='Cumplimiento cliente'
VT11_CUMPLE_CLIENTE['UNIDAD']='%'
VT11_CUMPLE_CLIENTE['cumple_cliente']=VT11_CUMPLE_CLIENTE['cumple_cliente']*100

In [25]:
 #VT11_CUMPLE_CLIENTE
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_GENERICA=VT11_CUMPLE_CLIENTE
VT11_GENERICA['DIA_SEMANA']=VT11_GENERICA.inPlanCarg.dt.dayofweek
VT11_GENERICA['DIA_MES']=VT11_GENERICA.inPlanCarg.dt.day
VT11_GENERICA['ANIO']=VT11_GENERICA.inPlanCarg.dt.year
VT11_GENERICA['DIA_ANIO']=VT11_GENERICA.inPlanCarg.dt.dayofyear
VT11_GENERICA['NO_SEMANA']=VT11_GENERICA.inPlanCarg.dt.weekofyear
VT11_GENERICA['MES']=VT11_GENERICA.inPlanCarg.dt.month


 #VT11_CUMPLE_CLIENTE
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['NO_SEMANA']==VT11_GENERICA.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_GENERICA['DIA_SEMANA']<=VT11_GENERICA.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['cumple_cliente'].mean()
    VT11_GENERICA.loc[row_index, 'cumple_cliente_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

 #VT11_CUMPLE_CLIENTE
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['MES']==VT11_GENERICA.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_GENERICA['DIA_MES']<=VT11_GENERICA.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['cumple_cliente'].mean()
    VT11_GENERICA.loc[row_index, 'cumple_cliente_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

     #VT11_CUMPLE_CLIENTE
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['ANIO']==VT11_GENERICA.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_GENERICA['DIA_ANIO']<=VT11_GENERICA.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['cumple_cliente'].mean()
    VT11_GENERICA.loc[row_index, 'cumple_cliente_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    


 #VT11_CUMPLE_CLIENTE
VT11_GENERICA['OBJETIVO_CUMPLE_CLIENTE']=VT11_GENERICA['OBJETIVO_CUMPLE_CLIENTE']*100
VT11_GENERICA['STATUS']=VT11_GENERICA['cumple_cliente']/VT11_GENERICA['OBJETIVO_CUMPLE_CLIENTE']*100
VT11_GENERICA['STATUS_SEMANAL']=VT11_GENERICA['cumple_cliente_SEMANAL']/VT11_GENERICA['OBJETIVO_CUMPLE_CLIENTE']*100
VT11_GENERICA['STATUS_MENSUAL']=VT11_GENERICA['cumple_cliente_MENSUAL']/VT11_GENERICA['OBJETIVO_CUMPLE_CLIENTE']*100
VT11_GENERICA['STATUS_ANUAL']=VT11_GENERICA['cumple_cliente_ANUAL']/VT11_GENERICA['OBJETIVO_CUMPLE_CLIENTE']*100
VT11_GENERICA.fillna(0, inplace=True)    

VT11_CUMPLE_CLIENTE_SEMANA= VT11_GENERICA

In [26]:
 #Cumple_tiempo_carga
VT11_CUMPLE_TIEMPO_CARGA=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['Cumple_tiempo_carga','OBJETIVO_CUMP_T_CARAGA']].mean()
VT11_CUMPLE_TIEMPO_CARGA=VT11_CUMPLE_TIEMPO_CARGA.reset_index()
VT11_CUMPLE_TIEMPO_CARGA['INDICADOR']='Cumplimiento Tiempo de Carga'
VT11_CUMPLE_TIEMPO_CARGA['UNIDAD']='%'
VT11_CUMPLE_TIEMPO_CARGA['Cumple_tiempo_carga']=VT11_CUMPLE_TIEMPO_CARGA['Cumple_tiempo_carga']*100

In [27]:
 #VT11_CUMPLE_TIEMPO_CARGA
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_GENERICA=VT11_CUMPLE_TIEMPO_CARGA
VT11_GENERICA['DIA_SEMANA']=VT11_GENERICA.inPlanCarg.dt.dayofweek
VT11_GENERICA['DIA_MES']=VT11_GENERICA.inPlanCarg.dt.day
VT11_GENERICA['ANIO']=VT11_GENERICA.inPlanCarg.dt.year
VT11_GENERICA['DIA_ANIO']=VT11_GENERICA.inPlanCarg.dt.dayofyear
VT11_GENERICA['NO_SEMANA']=VT11_GENERICA.inPlanCarg.dt.weekofyear
VT11_GENERICA['MES']=VT11_GENERICA.inPlanCarg.dt.month


 #VT11_CUMPLE_TIEMPO_CARGA
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['NO_SEMANA']==VT11_GENERICA.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_GENERICA['DIA_SEMANA']<=VT11_GENERICA.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_tiempo_carga'].mean()
    VT11_GENERICA.loc[row_index, 'Cumple_tiempo_carga_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

 #VT11_CUMPLE_TIEMPO_CARGA
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['MES']==VT11_GENERICA.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_GENERICA['DIA_MES']<=VT11_GENERICA.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_tiempo_carga'].mean()
    VT11_GENERICA.loc[row_index, 'cumple_cliente_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

     #VT11_CUMPLE_TIEMPO_CARGA
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['ANIO']==VT11_GENERICA.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_GENERICA['DIA_ANIO']<=VT11_GENERICA.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Cumple_tiempo_carga'].mean()
    VT11_GENERICA.loc[row_index, 'Cumple_tiempo_carga_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    


 #VT11_CUMPLE_TIEMPO_CARGA
VT11_GENERICA['OBJETIVO_CUMP_T_CARAGA']=VT11_GENERICA['OBJETIVO_CUMP_T_CARAGA']*100
VT11_GENERICA['STATUS']=VT11_GENERICA['Cumple_tiempo_carga']/VT11_GENERICA['OBJETIVO_CUMP_T_CARAGA']*100
VT11_GENERICA['STATUS_SEMANAL']=VT11_GENERICA['Cumple_tiempo_carga_SEMANAL']/VT11_GENERICA['OBJETIVO_CUMP_T_CARAGA']*100
VT11_GENERICA['STATUS_MENSUAL']=VT11_GENERICA['cumple_cliente_MENSUAL']/VT11_GENERICA['OBJETIVO_CUMP_T_CARAGA']*100
VT11_GENERICA['STATUS_ANUAL']=VT11_GENERICA['Cumple_tiempo_carga_ANUAL']/VT11_GENERICA['OBJETIVO_CUMP_T_CARAGA']*100
VT11_GENERICA.fillna(0, inplace=True)    

VT11_CUMPLE_TIEMPO_CARGA_SEMANAL=VT11_GENERICA 

In [28]:
#Total de toneladas métricas producidas
VT11_TONELADAS_PRODUCIDAS=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['pESO_rEAL']].sum()
VT11_TONELADAS_PRODUCIDAS=VT11_TONELADAS_PRODUCIDAS.reset_index()
VT11_TONELADAS_PRODUCIDAS['INDICADOR']='Total de toneladas métricas producidas'
VT11_TONELADAS_PRODUCIDAS['UNIDAD']='Ton'
VT11_TONELADAS_PRODUCIDAS['pESO_rEAL']=VT11_TONELADAS_PRODUCIDAS['pESO_rEAL']/1000

In [29]:
 #VT11_TONELADAS_PRODUCIDAS
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_GENERICA=VT11_TONELADAS_PRODUCIDAS
VT11_GENERICA['DIA_SEMANA']=VT11_GENERICA.inPlanCarg.dt.dayofweek
VT11_GENERICA['DIA_MES']=VT11_GENERICA.inPlanCarg.dt.day
VT11_GENERICA['ANIO']=VT11_GENERICA.inPlanCarg.dt.year
VT11_GENERICA['DIA_ANIO']=VT11_GENERICA.inPlanCarg.dt.dayofyear
VT11_GENERICA['NO_SEMANA']=VT11_GENERICA.inPlanCarg.dt.weekofyear
VT11_GENERICA['MES']=VT11_GENERICA.inPlanCarg.dt.month


 #VT11_TONELADAS_PRODUCIDAS
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['NO_SEMANA']==VT11_GENERICA.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_GENERICA['DIA_SEMANA']<=VT11_GENERICA.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['pESO_rEAL'].sum()
    VT11_GENERICA.loc[row_index, 'Total_toneladas_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

 #VT11_TONELADAS_PRODUCIDAS
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['MES']==VT11_GENERICA.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_GENERICA['DIA_MES']<=VT11_GENERICA.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['pESO_rEAL'].sum()
    VT11_GENERICA.loc[row_index, 'Total_toneladas_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

     #VT11_TONELADAS_PRODUCIDAS
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['ANIO']==VT11_GENERICA.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_GENERICA['DIA_ANIO']<=VT11_GENERICA.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['pESO_rEAL'].sum()
    VT11_GENERICA.loc[row_index, 'Total_toneladas_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    


 #VT11_TONELADAS_PRODUCIDAS
VT11_GENERICA['OBJETIVO_CUMPLE_CLIENTE']=0
VT11_GENERICA['STATUS']=0
VT11_GENERICA['STATUS_SEMANAL']=0
VT11_GENERICA['STATUS_MENSUAL']=0
VT11_GENERICA['STATUS_ANUAL']=0
VT11_GENERICA.fillna(0, inplace=True)    

VT11_TONELADAS_PRODUCIDAS_SEMANAL=VT11_GENERICA 

In [30]:
#Total Camiones Atendido
VT11_CAMIONES_ATENDIDOS=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['transporte']].count()
VT11_CAMIONES_ATENDIDOS=VT11_CAMIONES_ATENDIDOS.reset_index()
VT11_CAMIONES_ATENDIDOS['INDICADOR']='Total Camiones Atendidos'
VT11_CAMIONES_ATENDIDOS['UNIDAD']=''


In [31]:
 #VT11_CAMIONES_ATENDIDOS
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_GENERICA=VT11_CAMIONES_ATENDIDOS
VT11_GENERICA['DIA_SEMANA']=VT11_GENERICA.inPlanCarg.dt.dayofweek
VT11_GENERICA['DIA_MES']=VT11_GENERICA.inPlanCarg.dt.day
VT11_GENERICA['ANIO']=VT11_GENERICA.inPlanCarg.dt.year
VT11_GENERICA['DIA_ANIO']=VT11_GENERICA.inPlanCarg.dt.dayofyear
VT11_GENERICA['NO_SEMANA']=VT11_GENERICA.inPlanCarg.dt.weekofyear
VT11_GENERICA['MES']=VT11_GENERICA.inPlanCarg.dt.month


 #VT11_CAMIONES_ATENDIDOS
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['NO_SEMANA']==VT11_GENERICA.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_GENERICA['DIA_SEMANA']<=VT11_GENERICA.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['transporte'].sum()
    VT11_GENERICA.loc[row_index, 'CAMIONES_ATENDIDOS_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

 #VT11_CAMIONES_ATENDIDOS
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['MES']==VT11_GENERICA.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_GENERICA['DIA_MES']<=VT11_GENERICA.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['transporte'].sum()
    VT11_GENERICA.loc[row_index, 'CAMIONES_ATENDIDOS_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

     #VT11_CAMIONES_ATENDIDOS
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['ANIO']==VT11_GENERICA.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_GENERICA['DIA_ANIO']<=VT11_GENERICA.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['transporte'].sum()
    VT11_GENERICA.loc[row_index, 'CAMIONES_ATENDIDOS_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    


 #VT11_CAMIONES_ATENDIDOS
VT11_GENERICA['OBJETIVO_CUMPLE_CLIENTE']=0
VT11_GENERICA['STATUS']=0
VT11_GENERICA['STATUS_SEMANAL']=0
VT11_GENERICA['STATUS_MENSUAL']=0
VT11_GENERICA['STATUS_ANUAL']=0
VT11_GENERICA.fillna(0, inplace=True)    

VT11_CAMIONES_ATENDIDOS_SEMANAL=VT11_GENERICA 

In [32]:
# Convert account_opened to datetime
vt11['Plan_T_Car'] = pd.to_datetime(vt11['Plan_T_Car'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 
# Convert account_opened to datetime
vt11['Real_T_Car'] = pd.to_datetime(vt11['Real_T_Car'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce' ) 

In [33]:
vt11_Demora_carga=vt11['Plan_T_Car'] -  vt11['Real_T_Car'] 
vt11["Demora_tiempo_carga"]=abs(vt11_Demora_carga.dt.total_seconds())

In [34]:
#          OPE_INDICADORES_PLANTA.[Plan T Car] Plan_T_Car,
#          OPE_INDICADORES_PLANTA.[Real T Car] Real_T_Car,
# vt11.iloc[: ,[-1]]

for x,y in enumerate(vt11['Plan_T_Car']):
    if vt11.iloc[x,17]>vt11.iloc[x,18]:
        vt11.iloc[x,-1]=np.nan


In [35]:
#DIFERENCIA_CUMPLE_CLIENTE
VT11_DEMORA_TIEMPO_CARGA=vt11.groupby(['inPlanCarg','PLANTA','PAIS'])[['Demora_tiempo_carga','OBJETIVO_DEMORA_PROME_T_CARGA']].sum()
VT11_DEMORA_TIEMPO_CARGA=VT11_DEMORA_TIEMPO_CARGA.reset_index()
VT11_DEMORA_TIEMPO_CARGA['INDICADOR']='Demora en el tiempo de carga'
VT11_DEMORA_TIEMPO_CARGA['UNIDAD']='Minutos'
VT11_DEMORA_TIEMPO_CARGA['Demora_tiempo_carga']=VT11_DEMORA_TIEMPO_CARGA['Demora_tiempo_carga']/60
VT11_DEMORA_TIEMPO_CARGA['OBJETIVO_DEMORA_PROME_T_CARGA']=VT11_DEMORA_TIEMPO_CARGA['OBJETIVO_DEMORA_PROME_T_CARGA']/60

In [36]:
 #VT11_DEMORA_TIEMPO_CARGA
# df['rainfall'].rolling(min_periods=7, window=7).sum()
VT11_GENERICA=VT11_DEMORA_TIEMPO_CARGA
VT11_GENERICA['DIA_SEMANA']=VT11_GENERICA.inPlanCarg.dt.dayofweek
VT11_GENERICA['DIA_MES']=VT11_GENERICA.inPlanCarg.dt.day
VT11_GENERICA['ANIO']=VT11_GENERICA.inPlanCarg.dt.year
VT11_GENERICA['DIA_ANIO']=VT11_GENERICA.inPlanCarg.dt.dayofyear
VT11_GENERICA['NO_SEMANA']=VT11_GENERICA.inPlanCarg.dt.weekofyear
VT11_GENERICA['MES']=VT11_GENERICA.inPlanCarg.dt.month


 #VT11_DEMORA_TIEMPO_CARGA
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['NO_SEMANA']==VT11_GENERICA.loc[row_index ,'NO_SEMANA' ]
    v_Dia_semana=VT11_GENERICA['DIA_SEMANA']<=VT11_GENERICA.loc[row_index ,'DIA_SEMANA' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Demora_tiempo_carga'].count()
    VT11_GENERICA.loc[row_index, 'Demora_tiempo_carga_SEMANAL']=DEMORA_INICIO_CLIENTE_SEMANAL

 #VT11_DEMORA_TIEMPO_CARGA
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['MES']==VT11_GENERICA.loc[row_index ,'MES' ]
    v_Dia_semana=VT11_GENERICA['DIA_MES']<=VT11_GENERICA.loc[row_index ,'DIA_MES' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Demora_tiempo_carga'].mean()
    VT11_GENERICA.loc[row_index, 'Demora_tiempo_carga_MENSUAL']=DEMORA_INICIO_CLIENTE_SEMANAL

     #VT11_DEMORA_TIEMPO_CARGA
for  row_index,row  in VT11_GENERICA.iterrows():
    DEMORA_INICIO_CLIENTE_SEMANAL=0
    v_planta=VT11_GENERICA['PLANTA']== VT11_GENERICA.loc[row_index ,'PLANTA' ]
    v_pais=VT11_GENERICA['PAIS']==VT11_GENERICA.loc[row_index ,'PAIS' ]
    v_Nsemana=VT11_GENERICA['ANIO']==VT11_GENERICA.loc[row_index ,'ANIO' ]
    v_Dia_semana=VT11_GENERICA['DIA_ANIO']<=VT11_GENERICA.loc[row_index ,'DIA_ANIO' ]
    DEMORA_INICIO_CLIENTE_SEMANAL=VT11_GENERICA[v_planta & v_pais & v_Nsemana & v_Dia_semana]['Demora_tiempo_carga'].mean()
    VT11_GENERICA.loc[row_index, 'Demora_tiempo_carga_ANUAL']=DEMORA_INICIO_CLIENTE_SEMANAL
VT11_DEMORA_INICIO_CLIENTE_semana.fillna(0, inplace=True)    



 #VT11_CUMPLE_TIEMPO_CARGA
VT11_GENERICA['OBJETIVO_CUMP_T_CARAGA']=VT11_GENERICA['OBJETIVO_DEMORA_PROME_T_CARGA']*100
VT11_GENERICA['STATUS']=VT11_GENERICA['Demora_tiempo_carga']/VT11_GENERICA['OBJETIVO_DEMORA_PROME_T_CARGA']*100
VT11_GENERICA['STATUS_SEMANAL']=VT11_GENERICA['Demora_tiempo_carga_SEMANAL']/VT11_GENERICA['OBJETIVO_DEMORA_PROME_T_CARGA']*100
VT11_GENERICA['STATUS_MENSUAL']=VT11_GENERICA['Demora_tiempo_carga_MENSUAL']/VT11_GENERICA['OBJETIVO_DEMORA_PROME_T_CARGA']*100
VT11_GENERICA['STATUS_ANUAL']=VT11_GENERICA['Demora_tiempo_carga_ANUAL']/VT11_GENERICA['OBJETIVO_DEMORA_PROME_T_CARGA']*100
VT11_GENERICA.fillna(0, inplace=True)    

VT11_DEMORA_TIEMPO_CARGA_SEMANAL=VT11_GENERICA 

In [37]:
VT11_OTIF_FINAL=VT11_OTIF_semana[['inPlanCarg','PLANTA','PAIS','INDICADOR','UNIDAD','OTIF',
                  'OBJETIVO_OTIF','STATUS','OTIF_SEMANAL','OBJETIVO_OTIF',
                 'STATUS_SEMANAL','OTIF_MENSUL','OBJETIVO_OTIF','STATUS_MENSUAL',
                 'OTIF_ANIO','OBJETIVO_OTIF','STATUS_ANUAL']]


In [38]:
VT11_DEMORA_INICIO_CLIENTE_semana['STATUS' ]=0 
VT11_DEMORA_INICIO_CLIENTE_semana['STATUS_SEMANAL' ]=0 
VT11_DEMORA_INICIO_CLIENTE_semana['STATUS_MENSUAL' ]=0 
VT11_DEMORA_INICIO_CLIENTE_semana['STATUS_ANUAL' ]=0 
VT11_DEMORA_INICIO_CLIENTE_FINAL=VT11_DEMORA_INICIO_CLIENTE_semana[['inPlanCarg','PLANTA','PAIS','INDICADOR','UNIDAD','DIFERENCIA_CUMPLE_CLIENTE',
                  'OBJETIVO_DEMORA_PROME_H_INI_CARGA','STATUS','DIFERENCIA_CUMPLE_CLIENTE_SEMANAL','OBJETIVO_DEMORA_PROME_H_INI_CARGA',
                 'STATUS_SEMANAL','DIFERENCIA_CUMPLE_CLIENTE_MENSUAL','OBJETIVO_DEMORA_PROME_H_INI_CARGA','STATUS_MENSUAL',
                 'DIFERENCIA_CUMPLE_CLIENTE_ANUAL','OBJETIVO_DEMORA_PROME_H_INI_CARGA','STATUS_ANUAL']]

In [39]:
VT11_CUMPLE_INI_CARGA_FINAL=VT11_CUMPLE_INI_CARGA_semana[['inPlanCarg','PLANTA','PAIS','INDICADOR','UNIDAD','Cumple_inicio_carga',
                  'OBJETIVO_CUMP_H_INI_CARGA','STATUS','Cumplimiento Inicio de Carga_SEMANAL','OBJETIVO_CUMP_H_INI_CARGA',
                 'STATUS_SEMANAL','Cumplimiento Inicio de Carga_MENSUAL','OBJETIVO_CUMP_H_INI_CARGA','STATUS_MENSUAL',
                 'Cumplimiento Inicio de Carga_ANUAL','OBJETIVO_CUMP_H_INI_CARGA','STATUS_ANUAL']]


In [40]:
VT11_CUMPLE_CLIENTE_FINAL=VT11_CUMPLE_CLIENTE_SEMANA[['inPlanCarg','PLANTA','PAIS','INDICADOR','UNIDAD','cumple_cliente',
                  'OBJETIVO_CUMPLE_CLIENTE','STATUS','cumple_cliente_SEMANAL','OBJETIVO_CUMPLE_CLIENTE',
                 'STATUS_SEMANAL','cumple_cliente_MENSUAL','OBJETIVO_CUMPLE_CLIENTE','STATUS_MENSUAL',
                 'cumple_cliente_ANUAL','OBJETIVO_CUMPLE_CLIENTE','STATUS_ANUAL']]

In [41]:
VT11_CUMPLE_TIEMPO_CARGA_FINAL=VT11_CUMPLE_TIEMPO_CARGA_SEMANAL[['inPlanCarg','PLANTA','PAIS','INDICADOR','UNIDAD','Cumple_tiempo_carga',
                  'OBJETIVO_CUMP_T_CARAGA','STATUS','Cumple_tiempo_carga_SEMANAL','OBJETIVO_CUMP_T_CARAGA',
                 'STATUS_SEMANAL','cumple_cliente_MENSUAL','OBJETIVO_CUMP_T_CARAGA','STATUS_MENSUAL',
                 'Cumple_tiempo_carga_ANUAL','OBJETIVO_CUMP_T_CARAGA','STATUS_ANUAL']]

In [42]:
VT11_TONELADAS_PRODUCIDAS_FINAL=VT11_TONELADAS_PRODUCIDAS_SEMANAL[['inPlanCarg','PLANTA','PAIS','INDICADOR','UNIDAD','pESO_rEAL',
                  'OBJETIVO_CUMPLE_CLIENTE','STATUS','Total_toneladas_SEMANAL','OBJETIVO_CUMPLE_CLIENTE',
                 'STATUS_SEMANAL','Total_toneladas_MENSUAL','OBJETIVO_CUMPLE_CLIENTE','STATUS_MENSUAL',
                 'Total_toneladas_ANUAL','OBJETIVO_CUMPLE_CLIENTE','STATUS_ANUAL']]

In [43]:
VT11_CAMIONES_ATENDIDOS_FINAL=VT11_CAMIONES_ATENDIDOS_SEMANAL[['inPlanCarg','PLANTA','PAIS','INDICADOR','UNIDAD','transporte',
                  'OBJETIVO_CUMPLE_CLIENTE','STATUS','CAMIONES_ATENDIDOS_SEMANAL','OBJETIVO_CUMPLE_CLIENTE',
                 'STATUS_SEMANAL','CAMIONES_ATENDIDOS_MENSUAL','OBJETIVO_CUMPLE_CLIENTE','STATUS_MENSUAL',
                 'CAMIONES_ATENDIDOS_ANUAL','OBJETIVO_CUMPLE_CLIENTE','STATUS_ANUAL']]

In [44]:
VT11_DEMORA_TIEMPO_CARGA_FINAL=VT11_DEMORA_TIEMPO_CARGA_SEMANAL[['inPlanCarg','PLANTA','PAIS','INDICADOR','UNIDAD','Demora_tiempo_carga',
                  'OBJETIVO_DEMORA_PROME_T_CARGA','STATUS','Demora_tiempo_carga_SEMANAL','OBJETIVO_DEMORA_PROME_T_CARGA',
                 'STATUS_SEMANAL','Demora_tiempo_carga_MENSUAL','OBJETIVO_DEMORA_PROME_T_CARGA','STATUS_MENSUAL',
                 'Demora_tiempo_carga_ANUAL','OBJETIVO_DEMORA_PROME_T_CARGA','STATUS_ANUAL']]

In [45]:
columns = ['FECHA_CARGA', 'PLANTA', 'PAIS','INDICADOR','UNIDAD',
           'DIARIO','OBJETIVO', 'STATUS',
           'SEMANAL','OBJETIVO_SEMANAL', 'STATUS_SEMANAL',
           'MENSUAL','OBJETIVO_MENSUAL', 'STATUS_MENSUAL',
           'ANUAL','OBJETIVO_ANUAL', 'STATUS_ANUAL']



In [46]:
VT11_OTIF_FINAL.columns=columns
VT11_OTIF_FINAL.columns=columns
VT11_DEMORA_INICIO_CLIENTE_FINAL.columns=columns
VT11_CUMPLE_INI_CARGA_FINAL.columns=columns
VT11_CUMPLE_CLIENTE_FINAL.columns=columns
VT11_CUMPLE_TIEMPO_CARGA_FINAL.columns=columns
VT11_TONELADAS_PRODUCIDAS_FINAL.columns=columns
VT11_CAMIONES_ATENDIDOS_FINAL.columns=columns
VT11_DEMORA_TIEMPO_CARGA_FINAL.columns=columns

#pd.concat([VT11_OTIF_FINAL, VT11_DEMORA_INICIO_CLIENTE_FINAL])

In [47]:
vt11_final=pd.concat([VT11_OTIF_FINAL, VT11_DEMORA_INICIO_CLIENTE_FINAL,VT11_CUMPLE_INI_CARGA_FINAL, VT11_CUMPLE_CLIENTE_FINAL,VT11_CUMPLE_TIEMPO_CARGA_FINAL,VT11_TONELADAS_PRODUCIDAS_FINAL, VT11_CAMIONES_ATENDIDOS_FINAL,VT11_DEMORA_TIEMPO_CARGA_FINAL], ignore_index=True)

In [48]:
vt11_final=vt11_final.round(2 )

In [49]:
vt11_final1=vt11_final.head(1)

In [50]:
vt11_final.loc[ : , 'DIARIO']=vt11_final['DIARIO'].astype('str')
vt11_final.loc[ : , 'OBJETIVO']=vt11_final['OBJETIVO'].astype('str')
vt11_final.loc[ : , 'STATUS']=vt11_final['STATUS'].astype('str')
vt11_final.loc[ : , 'SEMANAL']=vt11_final['SEMANAL'].astype('str')
vt11_final.loc[ : , 'OBJETIVO_SEMANAL']=vt11_final['OBJETIVO_SEMANAL'].astype('str')
vt11_final.loc[ : , 'STATUS_SEMANAL']=vt11_final['STATUS_SEMANAL'].astype('str')
vt11_final.loc[ : , 'MENSUAL']=vt11_final['MENSUAL'].astype('str')
vt11_final.loc[ : , 'OBJETIVO_MENSUAL']=vt11_final['OBJETIVO_MENSUAL'].astype('str')
vt11_final.loc[ : , 'STATUS_MENSUAL']=vt11_final['STATUS_MENSUAL'].astype('str')
vt11_final.loc[ : , 'ANUAL']=vt11_final['ANUAL'].astype('str')
vt11_final.loc[ : , 'OBJETIVO_ANUAL']=vt11_final['OBJETIVO_ANUAL'].astype('str')
vt11_final.loc[ : , 'STATUS_ANUAL']=vt11_final['STATUS_ANUAL'].astype('str')

In [51]:
vt11_final.loc[ : , 'DIARIO']=vt11_final['DIARIO'].replace('inf','0.0')
vt11_final.loc[ : , 'OBJETIVO']=vt11_final['OBJETIVO'].replace('inf','0.0')
vt11_final.loc[ : , 'STATUS']=vt11_final['STATUS'].replace('inf','0.0')
vt11_final.loc[ : , 'SEMANAL']=vt11_final['SEMANAL'].replace('inf','0.0')
vt11_final.loc[ : , 'OBJETIVO_SEMANAL']=vt11_final['OBJETIVO_SEMANAL'].replace('inf','0.0')
vt11_final.loc[ : , 'STATUS_SEMANAL']=vt11_final['STATUS_SEMANAL'].replace('inf','0.0')
vt11_final.loc[ : , 'MENSUAL']=vt11_final['MENSUAL'].replace('inf','0.0')
vt11_final.loc[ : , 'OBJETIVO_MENSUAL']=vt11_final['OBJETIVO_MENSUAL'].replace('inf','0.0')
vt11_final.loc[ : , 'STATUS_MENSUAL']=vt11_final['STATUS_MENSUAL'].replace('inf','0.0')
vt11_final.loc[ : , 'ANUAL']=vt11_final['ANUAL'].replace('inf','0.0')
vt11_final.loc[ : , 'OBJETIVO_ANUAL']=vt11_final['OBJETIVO_ANUAL'].replace('inf','0.0')
vt11_final.loc[ : , 'STATUS_ANUAL']=vt11_final['STATUS_ANUAL'].replace('inf','0.0')

In [52]:
datos=[]
for i in vt11_final.index:
        datos.append(vt11_final.iloc[i,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]].tolist())
   

In [53]:
#cursor.executemany("insert into [dbo].[VT11_FINAL]  values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",datos)
#cnxn.commit()

cursor.execute(''' truncate table [dbo].[VT11_tmp_FINAL] ''')

In [54]:
for row in datos:
    cursor.execute("insert into [dbo].[VT11_tmp_FINAL]  values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",row)
    cnxn.commit()    

In [55]:
cursor.execute(''' truncate table [dbo].[VT11_FINAL]  ''')
cnxn.commit()  
cursor.execute(''' insert into [dbo].[VT11_FINAL]     
                    select * from [dbo].[VT11_tmp_FINAL]  ''')
cnxn.commit()  
